In [1]:
from shapely.geometry import Point, Polygon
import fiona 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%run ../python_files/data_cleaning

import warnings 
warnings.filterwarnings('ignore')

In [2]:
#import data
wards = gpd.read_file("../data/google_maps/gis/Wards.kml", driver='KML')
blocks_gis = gpd.read_file("../data/eviction_lab/gis/block-groups.geojson")
census_tracts_gis = gpd.read_file("../data/open_data/gis/Census_Tracts_in_2010.kml", driver='KML')
puds_gis = gpd.read_file("../data/open_data/gis/Planned_Unit_Development_PUDs.kml", driver='KML')
tifs_gis = gpd.read_file("../data/open_data/gis/Tax_Increment_Financing_TIF_areas.kml", driver='KML')
evict_data = pd.read_csv("../data/eviction_lab/csv/tracts.csv")
blocks = pd.read_csv("../data/eviction_lab/csv/block-groups.csv")
census_tracts = pd.read_csv('../data/open_data/csv/Census_Tracts_in_2010.csv')
puds = pd.read_csv('../data/open_data/csv/Planned_Unit_Development_PUDs.csv') 
tifs = pd.read_csv("../data/open_data/csv/Tax_Increment_Financing_TIF_areas.csv")
puds_xwalk = pd.read_csv('../data/grassroots_dc/csv/zoning_crosswalk.csv')
puds_gr = pd.read_csv('../data/grassroots_dc/csv/Grassroots PUD Data_njb - PUDs.csv')

In [3]:
#make blocks
blocks16 = clean_eviction_data(blocks)
blocks = make_eviction_df(blocks_gis, blocks16)
blocks = add_gis_ft1(wards, blocks, 'ward')

In [4]:
#make tracts
evict16 = clean_eviction_data(evict_data)
tracts = merge_tracts(census_tracts_gis, census_tracts)
evict = merge_eviction_data(tracts, evict16)
evict = add_gis_ft1(wards, evict, 'ward')

In [5]:
#make puds
puds = merge_gis(puds_gis, puds)
puds_gr = clean_grassroots_pud_data(puds_gr)
puds = add_pud_ft(puds, puds_xwalk, puds_gr)
puds = add_gis_ft2(blocks, puds, 'GEOID')

In [6]:
#make tifs
tifs = merge_gis(tifs_gis, tifs)
tifs = add_gis_ft2(blocks, tifs, 'GEOID')

In [8]:
#make master files
master_puds = create_master(puds, blocks)
export_master(master_puds, 'master_puds_blocks')
master_tifs = create_master(tifs, blocks)
export_master(master_tifs, 'master_tifs_blocks')

'exported master_tifs_blocks'